### <font color='blue'>Icoo Data Science</font>

# <font color='blue'>Mediphacos - Marketing Analytics</font>


### <font color='Brown'>Poc - Prova de Conceito</font>

### <font color='Brown'>Objetivo</font>
## <font color='blue'>Life Time Value Baseado em Segmentação de Clientes</font>
#### ♦ Etapa 01: Coleta de Dados
#### ♦ Etapa 02: Criar os segmentos de clientes
#### ♦ Etapa 03: Computar o fauramento por segmento
#### ♦ Etapa 04: Prever o Faturamento por segmento
#### ♦ Etapa 05: Encontrar o LTV baseado nessa previsão 

# <font color='Orange'>ETAPA 01: Criar os segmentos de clientes</font>

In [ ]:
# Ferramentas Data Science
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

from scipy import stats
from sklearn import linear_model as lm
from sklearn.metrics import r2_score
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
import statsmodels.formula.api as smf

from datetime import datetime, date
from pandasql import sqldf
import mysql.connector as mypy
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Ferramentas Gráficas
import plotly 
plotly.tools.set_credentials_file(username='ticula', api_key='suwl51AkciUx24bJuPjb')
plotly.offline.init_notebook_mode()

import plotly.graph_objs as go
import plotly.plotly as py

# Cufflinks wrapper on plotly
import cufflinks as cf

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot
cf.go_offline()

# Set global theme
# cf.set_config_file(world_readable=True, theme='pearl')
cf.set_config_file(offline=True, world_readable=True, theme='pearl')

In [ ]:
# Conexão com o banco
conn = mypy.connect(host='192.168.0.129', user='pipedrive', passwd='pipe@#951753#@drive', db='dw', charset='utf8')
cur = conn.cursor()

In [ ]:
# função para usar SQL
pysql = lambda query: sqldf(query, globals())

In [ ]:
# Extração dos dados
cur.execute('''
            SELECT 
            A.SK_ORG AS cliente_id,
            B.DT_DATA AS data_compra,
            TRUNCATE(SUM(A.VL_VENDA),2) AS valor_compra
            FROM FATO_FAT A 
                LEFT JOIN DIM_TEMPO B
                    ON A.SK_DATA = B.SK_DATA
            GROUP BY A.NK_FAT
            ''')

In [ ]:
dados = cur.fetchall()

In [ ]:
# Transformação em Tabela
dados = pd.DataFrame(dados, columns=[x[0] for x in cur.description])

In [ ]:
# Fechando a conexão com o banco
conn.close()
cur.close()

In [ ]:
dados.dtypes

In [ ]:
dados.shape

In [ ]:
dados.tail()

# <font color="orange">ETAPA 2: Segmentando Grupos de Clientes</font>

## <font color='Purple'>Uma Segmentacao Gerencial RFM  - (Recency, Frequency, Monetary Value)</font>
### <font color='Purple'>Duas Classificação. Uma ABC e outra LEAD SCORE. </font>

In [ ]:
# Fazendo uma cópia dos dados para presevar os dados originais
df = dados.copy()  

In [ ]:
# EXTRAINDO O ANO DA DATA DA COMPRA
df['ano_compra'] = df.data_compra.dt.year

In [ ]:
# EXTRAINDO DIAS DESDE A ÚLTIMA COMPRA. É NESSA COLUNA QUE SERÁ BASEADO A CRIAÇÃO DA RECÊNCIA DE COMPRA 
df['dias_desde'] = (date.today() - df.data_compra).dt.days

In [ ]:
# COMPUTANDO DADOS DE CLIENTES
clientes = pysql(''' 
        SELECT 
        cliente_id, 
        MIN(dias_desde) AS recencia,  
        COUNT(*) AS frequencia, 
        MAX(dias_desde) AS idade,
        AVG(valor_compra) AS media_compra,
        MAX(valor_compra) AS maior_compra,
        SUM(valor_compra) AS total_compra
        FROM df
        GROUP BY 1
        '''); clientes = clientes.round(2)

In [ ]:
# GERANDO ESTATÍSTICO DESCRITIVA. 
# ELA RESUME A TENDÊNCIA CENTRAL, A DISPERSÃO E A FORMA DE DISTRIBUIÇÃO. 
# HÁ VÁRIAS MANEIRAS DE FAZER A CLASSIFICAÇÃO. POR EXEMPLO: DISTRIBUIÇÃO DE FREQUÊNCIA, PERCENTIS, PESOS, ETC.
# A EXPERIENCIA DEMONSTRA QUE POUCOS CLIENTES, DE 10% A 20% DO TOTAL, SÃO DA CLASSE A, 
# ENQUANTO UM GRANDE DE QUANTIDADE, EM TORNO DE 50%, É DA CLASSE C E 30% A 40%, SÃO DA CLASSE B. 
clientes.iloc[:,1:].describe([.20, .30, .4, .50, .80])

In [ ]:
# FUNÇÃO PARA CLASSIFICAÇÃO ABC. 

def abc(x): 
    if x > clientes.media_compra.quantile(q = 0.8): 
        return str('A')
    elif (x <= clientes.media_compra.quantile(q = 0.8)) and (x > clientes.media_compra.quantile(q = 0.4)):
        return str('B')
    else:
        return str('C')

In [ ]:
# FUNCAO PARA CLASSIFICAÇÃO LEAD SCORING. 
def lead(x): 
    if x > 365*2: # Recência maior que 2 anos
        return str('I') # Inativo
    elif (x <= 365*2) and (x > 365): # Leads que não compram a mais do que 2 anos
        return str('F') # Frio
    elif (x <= 365) and (x > 180): # Leads que não compram a mais do que 1 ano
        return str('M') # Morno
    elif (x <= 180) and (x > 90): # Leads que não compram a mais do que 6 meses  
        return str('Q') # Quente
    elif x <= 90: # Leads que compraram a em 3 meses  
        return str('A') # Ativo
    else:
        return str('Nan') # Recência menor que seis meses

In [ ]:
clientes['abc'] = [abc for abc in map(abc, clientes.media_compra)]

In [ ]:
clientes['lead'] = [lead for lead in map(lead, clientes.recencia)]

In [ ]:
clientes['segmento'] = clientes.abc + clientes.lead

In [ ]:
clientes['media_compra'] = clientes.media_compra.round(decimals=2)

In [ ]:
grouped = clientes[['recencia','frequencia','media_compra', 'segmento']].groupby(['segmento']).agg(['max', 'min','std']).round(decimals=2)

In [ ]:
clientes.sort_values('recencia', ascending=True)

In [ ]:
clientes[['recencia', 'frequencia','media_compra','segmento']].set_index('segmento').iplot(
    kind='bar', 
    barmode='group',
    title = 'SEGMENTOS POR RECÊNCIA',
    yTitle = 'Recência',
    xTitle = 'Segmento'
)

In [ ]:
clientes[['frequencia', 'segmento']].set_index('segmento').iplot(
    kind='bar', 
    barmode='group',
    title = 'SEGMENTOS POR FREQUÊNCIA',
    yTitle = 'Frequência',
    xTitle = 'Segmento'
)

In [ ]:
clientes[['media_compra', 'segmento']].set_index('segmento').iplot(
    kind='bar', 
    barmode='group',
    title = 'SEGMENTOS POR TICKET MÉDIO',
    yTitle = 'Valor ',
    xTitle = 'Segmento'
)

# <font color='Orange'>ETAPA 3: Computar o fauramento por segmento</font>

In [ ]:
fat_2019 = pysql('''SELECT cliente_id, SUM(valor_compra) AS faturamento_2019 FROM df WHERE ano_compra = 2019 GROUP BY 1''').round(decimals=2)

In [ ]:
fat_2018 = pysql('''SELECT cliente_id, SUM(valor_compra) AS faturamento_2018 FROM df WHERE ano_compra = 2018 GROUP BY 1''').round(decimals=2)

In [ ]:
fat_2017 = pysql('''SELECT cliente_id, SUM(valor_compra) AS faturamento_2017 FROM df WHERE ano_compra = 2017 GROUP BY 1''').round(decimals=2)

In [ ]:
fat_2016 = pysql('''SELECT cliente_id, SUM(valor_compra) AS faturamento_2016 FROM df WHERE ano_compra = 2016 GROUP BY 1''').round(decimals=2)

In [ ]:
fat_2015 = pysql('''SELECT cliente_id, SUM(valor_compra) AS faturamento_2015 FROM df WHERE ano_compra = 2015 GROUP BY 1''').round(decimals=2)

In [ ]:
fat_2014 = pysql('''SELECT cliente_id, SUM(valor_compra) AS faturamento_2014 FROM df WHERE ano_compra = 2014 GROUP BY 1''').round(decimals=2)

In [ ]:
seg = clientes.merge(fat_2014, on='cliente_id')

In [ ]:
seg = seg.merge(fat_2015, on='cliente_id')

In [ ]:
seg = seg.merge(fat_2016, on='cliente_id')

In [ ]:
seg = seg.merge(fat_2017, on='cliente_id')

In [ ]:
seg = seg.merge(fat_2018, on='cliente_id')

In [ ]:
seg = seg.merge(fat_2019, on='cliente_id')

In [ ]:
seg.sortlevel(level=1, axis=1, inplace=True)

In [ ]:
seg[['recencia','frequencia','media_compra', 'segmento']].groupby(['segmento']).agg(['max', 'min','avg']).round(decimals=2)

In [ ]:
seg[['faturamento_2019', 'faturamento_2018', 'faturamento_2017', 'faturamento_2016', 'faturamento_2015', 'faturamento_2014', 'segmento']].set_index('segmento').iplot(
    kind='bar', 
    barmode='group',
    title = 'Faturamento por Segmento',
    yTitle = 'Faturamento',
    xTitle = 'segmento'
)

# <font color='Orange'>Etapa 04: Prever o Faturamento por segmento</font>

In [ ]:
df.tail()

## Construindo os Dados de Treino

In [ ]:
# # Computando faturamento gerado por clientes > 365 e Faturamento 2018 < 365
clientes_2017 = pysql('''SELECT cliente_id,
                               MIN(dias_desde -365) AS recencia ,
                               MAX(dias_desde - 365) AS idade,
                               COUNT(*) AS frequencia,
                               MIN(valor_compra) AS menor_compra,
                               AVG(valor_compra) AS media_compra,
                               MAX(valor_compra) AS maior_compra,
                               SUM(valor_compra) AS total_compra
                        FROM df
                        WHERE dias_desde > 365
                        GROUP BY 1
                        ORDER BY 1''').round(2)

In [ ]:
clientes_2017.tail()

In [ ]:
segmento = pysql(''' SELECT cliente_id, cluster FROM cluster2 ''')

In [ ]:
in_sample = clientes_2017.merge(segmento, on='cliente_id')

In [ ]:
in_sample.head()

In [ ]:
in_sample = in_sample.merge(fat_2018, on='cliente_id')

In [ ]:
in_sample.tail()

In [ ]:
# TRANSFORMANDO A COLUNA CLUSTER EM VARIÁVEIS DUMMY
le = LabelEncoder()

In [ ]:
in_sample['cluster'] = le.fit_transform(in_sample.cluster)

In [ ]:
in_sample.head()

In [ ]:
# RETIRANDO CLIENTES NAO IDENTIFICADOS
dados_treino = in_sample[in_sample['cliente_id'] != -1]

In [ ]:
dados_treino.tail()

## Construindo o Modelo Preditivo

### Matriz de Correlação

In [ ]:
# Normalizar os dados
dados_treino.iloc[:,:-1].corr()

In [ ]:
# Gerando a matriz
X = dados_treino.iloc[:,:-1].corr()

In [ ]:
# Gerando número de observações e variáveis
obsevacoes = len(dados_treino)
variaveis = dados_treino.columns[:-1]

In [ ]:
# Criando um Correlation Plot
def visualize_correlation_matrix(data, hurdle = 0.0):
    R = np.corrcoef(data, rowvar = 0)
    R[np.where(np.abs(R) < hurdle)] = 0.0
    heatmap = plt.pcolor(R, cmap = mpl.cm.coolwarm, alpha = 0.8)
    heatmap.axes.set_frame_on(False)
    heatmap.axes.set_yticks(np.arange(R.shape[0]) + 0.5, minor = False)
    heatmap.axes.set_xticks(np.arange(R.shape[1]) + 0.5, minor = False)
    heatmap.axes.set_xticklabels(variaveis, minor = False)
    plt.xticks(rotation=90)
    heatmap.axes.set_yticklabels(variaveis, minor = False)
    plt.tick_params(axis = 'both', which = 'both', bottom = 'off', top = 'off', left = 'off', right = 'off') 
    plt.colorbar()
    plt.show()

In [ ]:
# Visualizando o Plot
visualize_correlation_matrix(X, hurdle = 0.5)

In [ ]:
# Gerando eigenvalues e eigenvectors
corr = np.corrcoef(X, rowvar = 0)
eigenvalues, eigenvectors = np.linalg.eig(corr)

In [ ]:
print (eigenvalues)

In [ ]:
print (eigenvectors[:,2])

In [ ]:
print (variaveis[2], variaveis[5], variaveis[7])

### Normalizar Dados

In [ ]:
# Criando dados de teste
dados_teste = pysql('''SELECT cliente_id,
                               MIN(dias_desde) AS recencia ,
                               COUNT(*) AS frequencia,
                               AVG(valor_compra) AS media_compra
                        FROM df
                        WHERE ano_compra = 2018 AND cliente_id <> -1
                        GROUP BY 1
                        ORDER BY 1''').round(2)

In [ ]:
dados_teste1 = dados_teste.iloc[:,1:]; x_teste.head()

In [ ]:
X = dados_treino[['recencia','frequencia','media_compra']]
y = dados_treino['faturamento_2018'].values

# Normalziando Dados
Normalizer().fit_transform(X)
Normalizer().fit_transform(dados_teste1)

In [ ]:
# Datasets de treino e de teste

X_treino = X
y_treino = dados_treino['faturamento_2018'].values
x_teste = dados_teste1

# Criando o objeto linear regression
modelo = lm.LinearRegression(normalize = False, fit_intercept = True)

# Treinando o modelo com dados de treino e checando o score
modelo.fit(x_treino,y_treino)
modelo.score(x_treino, y_treino)


In [ ]:
valores_previstos = modelo.predict(x_teste)

In [ ]:
dados_teste['previsto'] = valores_previstos

In [ ]:
dados_teste

In [ ]:
for coef, var in sorted(zip(map(abs, modelo.coef_), dados_treino.columns[:-1]), reverse = True):
    print ("%6.3f %s" % (coef,var))

In [ ]:
standardization = StandardScaler()
Stand_coef_linear_reg = make_pipeline(standardization, modelo)

In [ ]:
Stand_coef_linear_reg.fit(X,y)
for coef, var in sorted(zip(map(abs, Stand_coef_linear_reg.steps[1][1].coef_), dados_treino.columns[:-1]), reverse = True):
    print ("%6.3f %s" % (coef,var))

In [ ]:
modelo2 = lm.LinearRegression(normalize = False, fit_intercept = True)

In [ ]:
def r2_est(X,y):
    return r2_score(y, modelo.fit(X,y).predict(X))

In [ ]:
print ('Baseline R2: %0.3f' %  r2_est(X,y))